In [1]:
import numpy as np
import os
import json
import pandas as pd

import tom
%reload_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
DATA_ROOT = '/beegfs/qx244/ds/openmic-2018/'

if not os.path.exists(DATA_ROOT):
    raise ValueError('Did you forget to set `DATA_ROOT`?')
    
with np.load(os.path.join(DATA_ROOT, 'openmic-2018.npz')) as OPENMIC:
    Y_true, Y_mask, sample_key = OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']
with open(os.path.join(DATA_ROOT, 'class-map.json'), 'r') as f:
    class_map = json.load(f)

In [3]:
# Let's split the data into the training and test set
# We use squeeze=True here to return a single array for each, rather than a full DataFrame
split_train = pd.read_csv(os.path.join(DATA_ROOT, 'partitions/split01_train.csv'),       
                          header=None, squeeze=True)
split_test = pd.read_csv(os.path.join(DATA_ROOT, 'partitions/split01_test.csv'), 
                         header=None, squeeze=True)
train_set = set(split_train)
test_set = set(split_test)

# These loops go through all sample keys, and save their row numbers
# to either idx_train or idx_test
#
# This will be useful in the next step for slicing the array data
idx_train, idx_test = [], []

for idx, n in enumerate(sample_key):
    if n in train_set:
        idx_train.append(idx)
    elif n in test_set:
        idx_test.append(idx)
    else:
        # This should never happen, but better safe than sorry.
        raise RuntimeError('Unknown sample key={}! Abort!'.format(sample_key[n]))
        
# Finally, cast the idx_* arrays to numpy structures
idx_train = np.asarray(idx_train)
idx_test = np.asarray(idx_test)

In [4]:
model = tom.construct_crnnL3_smp_tom()

In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
batch_size = 32
train_size = 2000
test_size = 100
train_gen = tom.MelGenerator(idx_train[:train_size], batch_size=batch_size, DATA_ROOT=DATA_ROOT)
test_gen = tom.MelGenerator(idx_test[:test_size], batch_size=batch_size, DATA_ROOT=DATA_ROOT)

In [21]:
model.fit_generator(
    generator=train_gen,
    epochs=2,
    validation_data=test_gen)

Epoch 1/2
63/63 [==============================] - 185s 3s/step - loss: 0.1802 - acc: 0.9348 - val_loss: 0.2774 - val_acc: 0.8680
Epoch 2/2
63/63 [==============================] - 176s 3s/step - loss: 0.1636 - acc: 0.9355 - val_loss: 0.2715 - val_acc: 0.8985
